In [ ]:
# import matplotlib
# import matplotlib.pyplot as plt
# plt.style.use(["default"])
# matplotlib.rc('lines',linewidth=3)
# matplotlib.rc('font',family='Nimbus Roman',weight='normal',size=20)
# c_frame=(0,0,0,.8)
# for tick in 'xtick','ytick':
#     matplotlib.rc(tick+'.major',width=1.5,size=8)
#     matplotlib.rc(tick+'.minor',width=1,size=4,visible=True)
#     matplotlib.rc(tick,color=c_frame,labelsize=18,direction='in')
# matplotlib.rc('xtick',top=True)
# matplotlib.rc('ytick',right=True)
# matplotlib.rc('axes',linewidth=1.5,edgecolor=c_frame,labelweight='normal')
# matplotlib.rc('grid',color=c_frame)
# matplotlib.rc('patch',edgecolor=c_frame)
# from matplotlib import rcParams
# rcParams['mathtext.fontset']='custom'
# rcParams['mathtext.rm']='Nimbus Roman'
# rcParams['mathtext.it']='Nimbus Roman'
# rcParams['mathtext.bf']='Nimbus Roman'

from pyhipp import plot
plot.runtime_config.use_stylesheets('mathtext-it')

In [ ]:
import sys
# sys.path.append("/home/cyma")
# sys.path.append("/home/cyma/illustris_python")
import illustris_python as il
import illustris_python.snapshot
import illustris_python.groupcat
# import illustris_python.add_func
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from matplotlib.ticker import AutoMinorLocator
from astropy.io import fits
import statistics

from pathlib import Path
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

class ColorSets:
    dark2 = plot.ColorSeq.predefined('dark2').get_rgba()
    tab10 = plot.ColorSeq.predefined('tab10').get_rgba()
    set1 = plot.ColorSeq.predefined('set1').get_rgba()
    set2 = plot.ColorSeq.predefined('set2').get_rgba()
    
    def __getitem__(self, name):
        if isinstance(name, tuple):
            return tuple(self[n] for n in name) 
        return getattr(self, name)

c_sets = ColorSets()

class NamedColors:
    k               = 'black'
    r               = c_sets.set1[0] 
    red             = c_sets.set1[0]
    pink            = '#c284b3'
    b               = c_sets.set1[1] 
    blue            = c_sets.set1[1]
    p               = c_sets.dark2[2] 
    purple          = c_sets.dark2[2]
    ly              = c_sets.dark2[5] 
    lightyellow     = c_sets.dark2[5]
    y               = c_sets.dark2[6] 
    yellow          = c_sets.dark2[6]
    g               = c_sets.dark2[0] 
    green           = c_sets.dark2[0]
    lg              = c_sets.set1[2] 
    lightgreen      = c_sets.set1[2]
    grey            = c_sets.dark2[-1]
    orange          = c_sets.dark2[1] 
    lightorange     = c_sets.set1[4]
    
    def __getitem__(self, name):
        if isinstance(name, tuple):
            return tuple(self[n] for n in name) 
        return getattr(self, name)
    

cs_named = NamedColors()

plot_data_dir = Path("./data")
out_fig_dir = Path("./figures")

xbins=np.arange(-1000,1000.1,5)
ybins=np.arange(-1000,1000.1,5)
xbins1=np.arange(-200,200.1,2)
ybins1=np.arange(-200,200.1,2)
xbins2=np.arange(-40,40.1,0.5)
ybins2=np.arange(-40,40.1,0.5)

In [ ]:
basePath='/data/public/TNG50-1/output/'
snapshot=99
a=1
fieldsA=['SubhaloFlag','SubhaloMassInRadType','SubhaloHalfmassRadType','SubhaloGrNr','SubhaloSFRinRad','SubhaloVel','SubhaloPos','SubhaloSpin']
subhalos=il.groupcat.loadSubhalos(basePath,snapshot,fieldsA)
x0=subhalos['SubhaloFlag']
x1=subhalos['SubhaloMassInRadType'][:,4]/0.6774
x2=subhalos['SubhaloHalfmassRadType'][:,4]/0.6774*a
x3=subhalos['SubhaloGrNr']
x4=subhalos['SubhaloSFRinRad']
x6=subhalos['SubhaloVel']
x7=subhalos['SubhaloPos']/0.6774*a
x8=subhalos['SubhaloSpin']
hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/galid.fits")
data=hdul[1].data
hdul.close()
idxA=data["galid"]
x1a=x1[idxA]
x2a=x2[idxA]
x3a=x3[idxA]
x4a=x4[idxA]
x6a=x6[idxA]
x7a=x7[idxA]
x8a=x8[idxA]
print(len(idxA))

In [ ]:
GroupFirstSub=il.groupcat.loadHalos(basePath,snapshot,'GroupFirstSub')[x3]
Group_R_Crit200=il.groupcat.loadHalos(basePath,snapshot,'Group_R_Crit200')[x3]/0.6774*a
Group_M_Crit200=il.groupcat.loadHalos(basePath,snapshot,'Group_M_Crit200')[x3]/0.6774
Group_R_Crit200a=Group_R_Crit200[idxA]
Group_M_Crit200a=Group_M_Crit200[idxA]
vr=np.sqrt(Group_M_Crit200a*Group_R_Crit200a)*207.41
print(np.nanmedian(x2a),np.nanmedian(Group_R_Crit200a))

In [ ]:
x1bh=subhalos['SubhaloMassInRadType'][:,5]/0.6774
x1bha=x1bh[idxA]

In [ ]:
idxAb=np.where((np.log10(x4a)>0.74878283*(np.log10(x1a)+10)-8.7082215)&(np.log10(x1bha)<-1.8))[0]
print(idxAb)
print(len(idxAb),np.nanmedian(Group_R_Crit200a[idxAb]))

In [ ]:
hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/galplane.fits")
data=hdul[1].data
hdul.close()
plane=data["plane"]

In [ ]:
wmf=np.zeros((len(idxA),len(xbins)-1,len(ybins)-1))
wvxf=np.zeros((len(idxA),len(xbins)-1,len(ybins)-1))
wvyf=np.zeros((len(idxA),len(xbins)-1,len(ybins)-1))
wm1f=np.zeros((len(idxA),len(xbins1)-1,len(ybins1)-1))
wvx1f=np.zeros((len(idxA),len(xbins1)-1,len(ybins1)-1))
wvy1f=np.zeros((len(idxA),len(xbins1)-1,len(ybins1)-1))
wm2f=np.zeros((len(idxA),len(xbins2)-1,len(ybins2)-1))
wvx2f=np.zeros((len(idxA),len(xbins2)-1,len(ybins2)-1))
wvy2f=np.zeros((len(idxA),len(xbins2)-1,len(ybins2)-1))
wme=np.zeros((len(idxA),len(xbins)-1,len(ybins)-1))
wvxe=np.zeros((len(idxA),len(xbins)-1,len(ybins)-1))
wvye=np.zeros((len(idxA),len(xbins)-1,len(ybins)-1))
wm1e=np.zeros((len(idxA),len(xbins1)-1,len(ybins1)-1))
wvx1e=np.zeros((len(idxA),len(xbins1)-1,len(ybins1)-1))
wvy1e=np.zeros((len(idxA),len(xbins1)-1,len(ybins1)-1))
wm2e=np.zeros((len(idxA),len(xbins2)-1,len(ybins2)-1))
wvx2e=np.zeros((len(idxA),len(xbins2)-1,len(ybins2)-1))
wvy2e=np.zeros((len(idxA),len(xbins2)-1,len(ybins2)-1))
for k in range(len(idxA)):
    spin=x8a[k]
    coordinate=x7a[k]
    velocity=x6a[k]
    hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/Alnilam{k}.fits")
    data=hdul[1].data
    hdul.close()
    coordinategasB=data["coordinategas"]
    hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/Alnitak{k}.fits")
    data=hdul[1].data
    hdul.close()
    velocitygasB=data["velocitygas"]
    hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/Betelgeuse{k}.fits")
    data=hdul[1].data
    hdul.close()
    massgasB=data["massgas"]
    print(k)
    gamar=plane[k]
    coordinategasB=il.add_func.func_face_onnew(coordinategasB,gamar,coordinate)
    velocitygasB=il.add_func.func_face_onnew(velocitygasB,gamar,velocity)+coordinategasB*0.06774
    print(k)
    idxC=np.where((coordinategasB[:,0]>-1000)&(coordinategasB[:,0]<1000)&(coordinategasB[:,1]>-1000)&
                  (coordinategasB[:,1]<1000)&(coordinategasB[:,2]>-10)&(coordinategasB[:,2]<10))[0]
    [wmf[k],
     wvxf[k],
     wvyf[k]]=il.add_func.multi_histogram2d(coordinategasB[idxC,0],coordinategasB[idxC,1],
                                            weights_list=[massgasB[idxC],massgasB[idxC]*velocitygasB[idxC,0],massgasB[idxC]*velocitygasB[idxC,1]],
                                            bins=[xbins,ybins])
    [wm1f[k],
     wvx1f[k],
     wvy1f[k]]=il.add_func.multi_histogram2d(coordinategasB[idxC,0],coordinategasB[idxC,1],
                                             weights_list=[massgasB[idxC],massgasB[idxC]*velocitygasB[idxC,0],massgasB[idxC]*velocitygasB[idxC,1]],
                                             bins=[xbins1,ybins1])
    [wm2f[k],
     wvx2f[k],
     wvy2f[k]]=il.add_func.multi_histogram2d(coordinategasB[idxC,0],coordinategasB[idxC,1],
                                             weights_list=[massgasB[idxC],massgasB[idxC]*velocitygasB[idxC,0],massgasB[idxC]*velocitygasB[idxC,1]],
                                             bins=[xbins2,ybins2])
    idxC=np.where((coordinategasB[:,0]>-1000)&(coordinategasB[:,0]<1000)&(coordinategasB[:,2]>-1000)&
                  (coordinategasB[:,2]<1000)&(coordinategasB[:,1]>-10)&(coordinategasB[:,1]<10))[0]
    [wme[k],
     wvxe[k],
     wvye[k]]=il.add_func.multi_histogram2d(coordinategasB[idxC,0],coordinategasB[idxC,2],
                                            weights_list=[massgasB[idxC],massgasB[idxC]*velocitygasB[idxC,0],massgasB[idxC]*velocitygasB[idxC,2]],
                                            bins=[xbins,ybins])
    [wm1e[k],
     wvx1e[k],
     wvy1e [k]]=il.add_func.multi_histogram2d(coordinategasB[idxC,0],coordinategasB[idxC,2],
                                             weights_list=[massgasB[idxC],massgasB[idxC]*velocitygasB[idxC,0],massgasB[idxC]*velocitygasB[idxC,2]],
                                             bins=[xbins1,ybins1])
    [wm2e[k],
     wvx2e[k],
     wvy2e[k]]=il.add_func.multi_histogram2d(coordinategasB[idxC,0],coordinategasB[idxC,2],
                                             weights_list=[massgasB[idxC],massgasB[idxC]*velocitygasB[idxC,0],massgasB[idxC]*velocitygasB[idxC,2]],
                                             bins=[xbins2,ybins2])
    print(k)

In [ ]:
wxf=np.nanmedian(wvxf[idxAb]/wmf[idxAb],axis=0).T
wyf=np.nanmedian(wvyf[idxAb]/wmf[idxAb],axis=0).T
wx1f=np.nanmedian(wvx1f[idxAb]/wm1f[idxAb],axis=0).T
wy1f=np.nanmedian(wvy1f[idxAb]/wm1f[idxAb],axis=0).T
wx2f=np.nanmedian(wvx2f[idxAb]/wm2f[idxAb],axis=0).T
wy2f=np.nanmedian(wvy2f[idxAb]/wm2f[idxAb],axis=0).T
wxe=np.nanmedian(wvxe[idxAb]/wme[idxAb],axis=0).T
wye=np.nanmedian(wvye[idxAb]/wme[idxAb],axis=0).T
wx1e=np.nanmedian(wvx1e[idxAb]/wm1e[idxAb],axis=0).T
wy1e=np.nanmedian(wvy1e[idxAb]/wm1e[idxAb],axis=0).T
wx2e=np.nanmedian(wvx2e[idxAb]/wm2e[idxAb],axis=0).T
wy2e=np.nanmedian(wvy2e[idxAb]/wm2e[idxAb],axis=0).T

In [ ]:
from astropy.table import Table
t=Table()
t['wxf']=wxf
t['wyf']=wyf
t.write("/home/cyma/Sagittarius/Nunkifaceon.fits",format='fits')
t=Table()
t['wx1f']=wx1f
t['wy1f']=wy1f
t.write("/home/cyma/Sagittarius/Nunkifaceon1.fits",format='fits')
t=Table()
t['wx2f']=wx2f
t['wy2f']=wy2f
t.write("/home/cyma/Sagittarius/Nunkifaceon2.fits",format='fits')
t=Table()
t['wxe']=wxe
t['wye']=wye
t.write("/home/cyma/Sagittarius/Nunkiedgeon.fits",format='fits')
t=Table()
t['wx1e']=wx1e
t['wy1e']=wy1e
t.write("/home/cyma/Sagittarius/Nunkiedgeon1.fits",format='fits')
t=Table()
t['wx2e']=wx2e
t['wy2e']=wy2e
t.write("/home/cyma/Sagittarius/Nunkiedgeon2.fits",format='fits')

### Plot fields

In [ ]:
hdul=fits.open(plot_data_dir/"Nunkifaceon.fits")
data=hdul[1].data
hdul.close()
wxf=data['wxf']
wyf=data['wyf']
hdul=fits.open(plot_data_dir/"Nunkifaceon1.fits")
data=hdul[1].data
hdul.close()
wx1f=data['wx1f']
wy1f=data['wy1f']
hdul=fits.open(plot_data_dir/"Nunkifaceon2.fits")
data=hdul[1].data
hdul.close()
wx2f=data['wx2f']
wy2f=data['wy2f']
hdul=fits.open(plot_data_dir/"Nunkiedgeon.fits")
data=hdul[1].data
hdul.close()
wxe=data['wxe']
wye=data['wye']
hdul=fits.open(plot_data_dir/"Nunkiedgeon1.fits")
data=hdul[1].data
hdul.close()
wx1e=data['wx1e']
wy1e=data['wy1e']
hdul=fits.open(plot_data_dir/"Nunkiedgeon2.fits")
data=hdul[1].data
hdul.close()
wx2e=data['wx2e']
wy2e=data['wy2e']

In [ ]:
flow_speedf=np.hypot(wxf,wyf)
flow_speed1f=np.hypot(wx1f,wy1f)
flow_speed2f=np.hypot(wx2f,wy2f)
normf=mcolors.Normalize(vmin=0,vmax=120)
flow_speede=np.hypot(wxe,wye)
flow_speed1e=np.hypot(wx1e,wy1e)
flow_speed2e=np.hypot(wx2e,wy2e)
norme=mcolors.Normalize(vmin=0,vmax=65)
theta=np.linspace(0,2*np.pi,1000)

In [ ]:
Rh0009=97.142426
RISM0009=28.56607261
RCGM0009=165.75405286
Rh0010=148.96107
RISM0010=41.47820459
RCGM0010=320.18500756
Rh0011=234.71773
RISM0011=53.13176842
RCGM0011=501.73132151
Rh0110=108.1714
RISM0110=19.29738728
RCGM0110=648.31024443
Rh0210=82.0893
RISM0210=9.42897784
RCGM0210=491.86039223

In [ ]:
fig, ax = plot.subplots(1, figsize=(2.25, .85), margin=[0.05, 0.05, 0.785, 0.02], layout='none')

sm = mpl.cm.ScalarMappable(cmap='rainbow', norm=normf)
fig.colorbar(sm, cax=ax._raw, 
             location='bottom', label=r'$v[{\rm km/s}]$')

plot.savefig(out_fig_dir/'streamline_fcb.pdf')

In [ ]:
fig, ax = plot.subplots(1, figsize=(2.25, .85), margin=[0.05, 0.05, 0.785, 0.02], layout='none')

sm = mpl.cm.ScalarMappable(cmap='rainbow', norm=norme)
fig.colorbar(sm, cax=ax._raw, 
             location='bottom', label=r'$v[{\rm km/s}]$')

plot.savefig(out_fig_dir/'streamline_ecb.pdf')

In [ ]:
fig, ax = plot.subplots(1, figsize=6., margin=[0.1, 0.1, 0.05, 0.05], layout='none')

strm= ax._raw.streamplot(xbins2[:-1]+0.25,ybins2[:-1]+0.25,wx2f,wy2f,
    color=flow_speed2f,cmap='rainbow',norm=normf,linewidth=1.,density=3,arrowsize=.8)

ax.lim([-39, 39], [-39, 39])#label(r'x\,[{\rm kpc}]', r'y\,[{\rm kpc}]')\    
ax._raw.set_aspect('equal')
ax._raw.grid(False)
ax._raw.set_facecolor('k')
ax._raw.xaxis.tick_top()
ax._raw.yaxis.tick_right()
ax._raw.tick_params(axis='both', color='w', width=1, length=5)
plot.savefig(out_fig_dir/'streamline_f3.pdf')

In [ ]:
fig, ax = plot.subplots(1, figsize=6., margin=[0.1, 0.1, 0.05, 0.05], layout='none')

strm= ax._raw.streamplot(xbins2[:-1]+0.25,ybins2[:-1]+0.25,wx2e,wy2e,color=flow_speed2e,
                         cmap='rainbow',norm=norme,linewidth=1.,density=3,arrowsize=.8)

ax.lim([-39, 39], [-39, 39])#label(r'x\,[{\rm kpc}]', r'y\,[{\rm kpc}]')\    
ax._raw.set_aspect('equal')
ax._raw.grid(False)
ax._raw.set_facecolor('k')
ax._raw.xaxis.tick_top()
ax._raw.yaxis.tick_right()
ax._raw.tick_params(axis='both', color='w', width=1, length=5)
plot.savefig(out_fig_dir/'streamline_e3.pdf')

In [ ]:
fig, ax = plot.subplots(1, figsize=6., margin=[0.05, 0.1, 0.1, 0.05], layout='none')

strm= ax._raw.streamplot(xbins1[:-1]+1,ybins1[:-1]+1,wx1f,wy1f,
                    color=flow_speed1f,cmap='rainbow',norm=normf,linewidth=1.,density=5,arrowsize=.8)
ax.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='grey',ls=(0,(2,1)),lw=1)

ax.lim([-199, 199], [-199, 199])#label(r'x\,[{\rm kpc}]', r'y\,[{\rm kpc}]')\    
ax._raw.set_aspect('equal')
ax._raw.grid(False)
ax._raw.set_facecolor('k')
ax._raw.yaxis.tick_right()
ax._raw.tick_params(axis='both', color='w', width=1, length=5)
plot.savefig(out_fig_dir/'streamline_f2.pdf')

In [ ]:
fig, ax = plot.subplots(1, figsize=6., margin=[0.05, 0.1, 0.1, 0.05], layout='none')

strm= ax._raw.streamplot(xbins1[:-1]+1,ybins1[:-1]+1,wx1e,wy1e,color=flow_speed1e,
                         cmap='rainbow',norm=norme,linewidth=1.,density=5,arrowsize=.8)
ax.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='grey',ls=(0,(2,1)),lw=1)

ax.lim([-199, 199], [-199, 199])#label(r'x\,[{\rm kpc}]', r'y\,[{\rm kpc}]')\    
ax._raw.set_aspect('equal')
ax._raw.grid(False)
ax._raw.set_facecolor('k')
ax._raw.yaxis.tick_right()
ax._raw.tick_params(axis='both', color='w', width=1, length=5)
plot.savefig(out_fig_dir/'streamline_e2.pdf')

In [ ]:
fig, ax = plot.subplots(1, figsize=12.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

strm=ax._raw.streamplot(xbins[:-1]+2.5,ybins[:-1]+2.5,wxf,wyf,
                        color=flow_speedf,cmap='rainbow',norm=normf,
                        linewidth=1., density=15, 
                        arrowsize=.8)
ax.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='grey',ls=(0,(2,1)),lw=1)

ax.label(r'x\,[{\rm kpc}]', r'y\,[{\rm kpc}]')\
    .lim([-675, 675], [-675, 675])
ax._raw.set_aspect('equal')
ax._raw.grid(False)
ax._raw.set_facecolor('k')
ax._raw.tick_params(axis='both', color='w', width=1, length=5)
plot.savefig(out_fig_dir/'streamline_f1.pdf')

In [ ]:
fig, ax = plot.subplots(1, figsize=12.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

strm=ax._raw.streamplot(
    xbins[:-1]+2.5,ybins[:-1]+2.5,wxe,wye,
    color=flow_speede,cmap='rainbow',norm=norme,linewidth=1.,density=15, arrowsize=.8
)
ax.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='grey',ls=(0,(2,1)),lw=1)

ax.label(r'x\,[{\rm kpc}]', r'z\,[{\rm kpc}]')\
    .lim([-675, 675], [-675, 675])
ax._raw.set_aspect('equal')
ax._raw.grid(False)
ax._raw.set_facecolor('k')
ax._raw.tick_params(axis='both', color='w', width=1, length=5)
plot.savefig(out_fig_dir/'streamline_e1.pdf')

In [ ]:
fig=plt.figure(figsize=(20,20))

ax=fig.add_subplot(211)
ax.set_title('Face-on')
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]',labelpad=-30)
ax.set_xlim([-1000,1000])
ax.set_ylim([-1000,1000])
ax.text(0.1,0.1,'a1',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=ax.transAxes)
strm=ax.streamplot(xbins[:-1]+2.5,ybins[:-1]+2.5,wxf,wyf,color=flow_speedf,cmap='jet',norm=normf,linewidth=1,density=5)
ax.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='black',linestyle='--',linewidth=1)
ax.text(0,Rh0010,'R$_{200}$',fontsize=15,ha='left',va='center',zorder=3)
ax.set_aspect('equal')

axins1=ax.inset_axes([-0.65,0.52,0.48,0.48])
axins1.set_xlim(-200,200)
axins1.set_ylim(-200,200)
axins1.text(0.1,0.1,'a2',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=axins1.transAxes)
axins1.streamplot(xbins1[:-1]+1,ybins1[:-1]+1,wx1f,wy1f,color=flow_speed1f,cmap='jet',norm=normf,linewidth=1,density=3)
axins1.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='black',linestyle='--',linewidth=1)
ax.indicate_inset_zoom(axins1,edgecolor="black")

axins2=axins1.inset_axes([0,-1.08,1,1])
axins2.set_xlim(-40,40)
axins2.set_ylim(-40,40)
axins2.text(0.1,0.1,'a3',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=axins2.transAxes)
axins2.streamplot(xbins2[:-1]+0.25,ybins2[:-1]+0.25,wx2f,wy2f,color=flow_speed2f,cmap='jet',norm=normf,linewidth=1,density=3)
axins1.indicate_inset_zoom(axins2,edgecolor="black")
cbar=fig.colorbar(strm.lines,ax=ax,aspect=20,pad=0.01)
cbar.set_label('v [km/s]')

ax=fig.add_subplot(212)
ax.set_title('Edge-on')
ax.set_xlabel('x [kpc]')
ax.set_ylabel('z [kpc]',labelpad=-30)
ax.set_xlim([-1000,1000])
ax.set_ylim([-1000,1000])
ax.text(0.1,0.1,'b1',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=ax.transAxes)
strm=ax.streamplot(xbins[:-1]+2.5,ybins[:-1]+2.5,wxe,wye,color=flow_speede,cmap='jet',norm=norme,linewidth=1,density=4)
ax.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='black',linestyle='--',linewidth=1)
ax.text(0,Rh0010,'R$_{200}$',fontsize=15,ha='left',va='center',zorder=3)
ax.set_aspect('equal')

axins1=ax.inset_axes([-0.65,0.52,0.48,0.48])
axins1.set_xlim(-200,200)
axins1.set_ylim(-200,200)
axins1.text(0.1,0.1,'b2',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=axins1.transAxes)
axins1.streamplot(xbins1[:-1]+1,ybins1[:-1]+1,wx1e,wy1e,color=flow_speed1e,cmap='jet',norm=norme,linewidth=1,density=2)
axins1.plot(Rh0010*np.cos(theta),Rh0010*np.sin(theta),c='black',linestyle='--',linewidth=1)
ax.indicate_inset_zoom(axins1,edgecolor="black")

axins2=axins1.inset_axes([0,-1.08,1,1])
axins2.set_xlim(-40,40)
axins2.set_ylim(-40,40)
axins2.text(0.1,0.1,'b3',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=axins2.transAxes)
axins2.streamplot(xbins2[:-1]+0.25,ybins2[:-1]+0.25,wx2e,wy2e,color=flow_speed2e,cmap='jet',norm=norme,linewidth=1,density=2)
axins1.indicate_inset_zoom(axins2,edgecolor="black")
cbar=fig.colorbar(strm.lines,ax=ax,aspect=20,pad=0.01)
cbar.set_label('v [km/s]')
plt.tight_layout()
fig.savefig(out_fig_dir/'streamline.pdf',bbox_inches='tight',pad_inches=0.5)